<a href="https://colab.research.google.com/github/arushiagrawal-123/DataSpark_gdg/blob/main/ml_prioritization_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving complaints_raw_3000.csv to complaints_raw_3000 (1).csv


In [ ]:
import pandas as pd

df = pd.read_csv("complaints_raw_3000.csv")
print(df.head())
print(df.columns)

  complaint_id issue_type                                        description  \
0        C0001    hygiene  Issue related to electricity reported near Hos...   
1        C0002       road     Issue related to garbage reported near Canteen   
2        C0003    leakage    Issue related to leakage reported near Hostel B   
3        C0004       road      Issue related to light reported near Hostel B   
4        C0005       road  Issue related to electricity reported near Hos...   

    latitude  longitude   location_name reported_at resolved_at    status  \
0  26.856053  80.944847         Library  2024-05-31  2024-07-15      open   
1  26.849164  80.953266        Hostel C  2024-01-18  2024-01-21      open   
2  26.849659  80.956782        Hostel C  2024-02-28         NaN  resolved   
3  26.842670  80.951013        Hostel C  2024-04-06  2024-04-21  resolved   
4  26.841612  80.943063  Academic Block  2024-04-08         NaN      open   

   repeat_count  unsafe_flag  
0             5          

In [ ]:
print(df.columns)



Index(['complaint_id', 'issue_type', 'description', 'latitude', 'longitude',
       'location_name', 'reported_at', 'resolved_at', 'status', 'repeat_count',
       'unsafe_flag'],
      dtype='object')


In [ ]:
df = df[['description', 'repeat_count', 'unsafe_flag']]
df.head()


,description,repeat_count,unsafe_flag
0,Issue related to electricity reported near Hos...,5,1
1,Issue related to garbage reported near Canteen,9,1
2,Issue related to leakage reported near Hostel B,3,1
3,Issue related to light reported near Hostel B,1,1
4,Issue related to electricity reported near Hos...,2,0


In [ ]:
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['clean_description'] = df['description'].apply(clean_text)
df.head()


,description,repeat_count,unsafe_flag,clean_description
0,Issue related to electricity reported near Hos...,5,1,issue related to electricity reported near hos...
1,Issue related to garbage reported near Canteen,9,1,issue related to garbage reported near canteen
2,Issue related to leakage reported near Hostel B,3,1,issue related to leakage reported near hostel b
3,Issue related to light reported near Hostel B,1,1,issue related to light reported near hostel b
4,Issue related to electricity reported near Hos...,2,0,issue related to electricity reported near hos...


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    max_features=3000,
    ngram_range=(1, 2),
    stop_words='english'
)

X_text = tfidf.fit_transform(df['clean_description'])


In [ ]:
df['repeat_count'] = pd.to_numeric(df['repeat_count'], errors='coerce').fillna(0)
df['unsafe_flag'] = pd.to_numeric(df['unsafe_flag'], errors='coerce').fillna(0)



In [ ]:
X_numeric = df[['repeat_count', 'unsafe_flag']].astype(float).values


In [ ]:
from scipy.sparse import hstack

X = hstack([X_text, X_numeric])


In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
df['cluster'] = kmeans.fit_predict(X)



In [ ]:
cluster_priority = (
    df.groupby('cluster')[['repeat_count', 'unsafe_flag']]
    .mean()
    .sum(axis=1)
    .sort_values()
)

print(cluster_priority)


cluster
0    2.478846
2    5.486842
1    8.483539
dtype: float64


In [ ]:
priority_map = {
    cluster_priority.index[0]: 'Low',
    cluster_priority.index[1]: 'Medium',
    cluster_priority.index[2]: 'High'
}

df['priority_label'] = df['cluster'].map(priority_map)


In [ ]:
df[['description', 'repeat_count', 'unsafe_flag', 'cluster', 'priority_label']].head()


,description,repeat_count,unsafe_flag,cluster,priority_label
0,Issue related to electricity reported near Hos...,5,1,2,Medium
1,Issue related to garbage reported near Canteen,9,1,1,High
2,Issue related to leakage reported near Hostel B,3,1,0,Low
3,Issue related to light reported near Hostel B,1,1,0,Low
4,Issue related to electricity reported near Hos...,2,0,0,Low
